In [1]:
import os
import cv2 
import numpy as np
#from ultralytics import YOLO

In [ ]:
# Load the model


In [ ]:
# Load the image
def get_img():
    import requests

    url = "https://img03.platesmania.com/221209/m/20496102.jpg"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        with open("img.jpg", "wb") as f:
            f.write(response.content)
        print("Image downloaded successfully!")
    else:
        print("Failed to download image:", response.status_code)

In [ ]:
def crop_image_with_mode(model_path, image_path, output_dir='cropped_outputs'):
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # model_path = 'COEN490/Koala/ALPR/YOLOv8n_TEST/my_project/exp1/weights/best.pt'
    model = YOLO(model_path)

    # Read the image using cv2
    # (Note: cv2 reads images in BGR format by default)
    image = cv2.imread(image_path)
    
    # Run inference (prediction) on the image
    # The results returned is a list of results (one per image in the batch)
    results = model.predict(source=image_path, conf=0.25, iou=0.45)
    
    for result in results:
        # The 'boxes' attribute contains the bounding boxes.
        # The .xyxy attribute returns the bounding boxes in [x1, y1, x2, y2] format.
        boxes = result.boxes.xyxy.cpu().numpy()  # Convert to NumPy array if on GPU
    
        # Loop through each detected bounding box
        for idx, box in enumerate(boxes):
            # Extract coordinates and cast them to integer
            x1, y1, x2, y2 = map(int, box)
            
            # Crop the image using the bounding box coordinates
            # Note: image is a NumPy array with shape (height, width, channels)
            cropped_image = image[y1:y2, x1:x2]
            # Define a filename for the cropped image
            output_path = os.path.join(output_dir, f'cropped_object_{idx+1}.jpg')
            
            # Save the cropped image using cv2.imwrite
            cv2.imwrite(output_path, cropped_image)
            print(f'Saved cropped image to {output_path}')
            

In [ ]:
mod_path = "/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/my_project/exp1/weights/best.pt"
image_path = '/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/datasets/other/img.jpg'
crop_image_with_mode(mod_path, image_path)

In [51]:
# find contours

def upscale_img(im):
    scale_factor = 5.0
    new_width = int(im.shape[1] * scale_factor)
    new_height = int(im.shape[0] * scale_factor)

    # Resize the image using cubic interpolation (better quality)
    upscaled_im = cv2.resize(im, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    return upscaled_im

def find_rect(cnt_points):
    """
    Returns a list of rectangles set as 4 points 
    """
    # first find the leftmost point out of all the contours
    
    biggest_rect = []
    for cnt in cnt_points: 
        leftmost = min(cnt, key=lambda p: p[0][0]*p[0][0] + p[0][1]*p[0][1])
        sorted_points = sorted(cnt, key=lambda p: (leftmost[0][0] - p[0][0]) * (leftmost[0][0] - p[0][0]) + (leftmost[0][1] - p[0][1]) * (leftmost[0][1] - p[0][1]), reverse=True)
        biggest_rect.append(np.array([leftmost, *sorted_points[:3]]))
    print("\nPrinting biggest_rect:")
    print(biggest_rect)
    return biggest_rect

def find_contours(image_path):
    """
        Finds the contours(edges) of cropped license plate
    """
    directory = "/nfs/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST"
    os.chdir(directory)

    im = cv2.imread(image_path)
    # im = upscale_img(im)
    assert im is not None, "file could not be read, check with os.path.exists()"
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(imgray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    blur = cv2.GaussianBlur(thresh,(5,5),0)
    th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filtering contours
    license_plate_contour = []
    for cnt in contours:
        # Get bounding rectangle
        x, y, w, h = cv2.boundingRect(cnt)
        
        # Define aspect ratio and size constraints
        aspect_ratio = w / float(h)
        if 1 < aspect_ratio < 6 and 70000 < cv2.contourArea(cnt):  
            print(f"\nContour area for:")
            print(cv2.contourArea(cnt))
            approx = cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), True)
            # Check if it's a quadrilateral (4 corners)
            print("Sides detected:")

            print(len(approx))

            if 3 < len(approx) < 7:
                print("Sides detected:")
                print(len(approx))
                license_plate_contour.append(approx)
                # break  # Assuming only one plate, stop searching

    # for cnt in contours:
    #     print(f"{cv2.contourArea(cnt)}")
    # Draw the detected license plate
    # if license_plate_contour is not None:
    
    cv2.drawContours(im, [license_plate_contour[1]], -1, (0, 255, 0), 3)
    
    rectangles_extracted = find_rect(license_plate_contour)
    warp_im = None
    for cnt in rectangles_extracted:
        is_rect, confidence = is_rectangle(cnt)
        print(f"{cnt}: is Rectangle? {is_rect}, Confidence Level: {confidence}")
        if confidence > 20: 
            cv2.drawContours(im, [cnt], -1, (0, 255, 0), 3)
            warp_im = straighten_image(cnt, im)
        
    # upscaled_image = upscale_baby(warp_im)
    print(license_plate_contour[0])
    
    print("Before saving image:")  
    print(os.listdir(directory))  

    cv2.imwrite("img_contour_out.jpg", im)
    cv2.imwrite("warp_im.jpg", warp_im)
    # cv2.imwrite("upscaled_img.jpg", upscaled_image)


In [52]:
image_path = '/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/Pipeline/test.jpg'
find_contours(image_path)


Contour area for:
77237.5
Sides detected:
11
Sides detected:
11

Contour area for:
146320.5
Sides detected:
22


IndexError: list index out of range

In [9]:
def is_rectangle(points, tolerance=5):
    """
    Checks if four given points form a rectangle and returns a confidence score.

    Parameters:
        points (list of lists or np.array): Four points as [[x1, y1], [x2, y2], [x3, y3], [x4, y4]].
        tolerance (float): Allowed deviation in degrees for right angles.

    Returns:
        (bool, float): (True if rectangle, False otherwise, confidence score in percentage)
    """
    print("Rectangle Structure")
    points = restructure_array(points)
    print(points)
    if len(points) != 4:
        return False, 0.0

    # Convert to numpy array
    points = np.array(points, dtype=np.float32)

    # Compute centroid
    centroid = np.mean(points, axis=0)

    # Sort points by angle around centroid (ensures consistent ordering)
    points = sorted(points, key=lambda p: np.arctan2(p[1] - centroid[1], p[0] - centroid[0]))

    # Compute vectors for each side
    vectors = [points[i] - points[(i + 1) % 4] for i in range(4)]

    # Compute edge lengths
    lengths = [np.linalg.norm(v) for v in vectors]

    # Check if opposite sides are approximately equal
    side_match = abs(lengths[0] - lengths[2]) < 1e-3 and abs(lengths[1] - lengths[3]) < 1e-3

    # Compute angles using dot product
    angles = []
    for i in range(4):
        v1 = vectors[i] / np.linalg.norm(vectors[i])  # Normalize
        v2 = vectors[(i + 1) % 4] / np.linalg.norm(vectors[(i + 1) % 4])
        dot_product = np.dot(v1, v2)
        angle = np.arccos(np.clip(dot_product, -1.0, 1.0)) * (180 / np.pi)
        angles.append(angle)

    # Check if all angles are approximately 90 degrees
    angle_match = all(abs(angle - 90) < tolerance for angle in angles)

    # Compute confidence score (100% if all angles are exactly 90)
    confidence = 100 - (sum(abs(angle - 90) for angle in angles) / 4)

    return side_match and angle_match, confidence

In [ ]:
points = [[0, 0], [4, 0], [4, 3], [0, 3]]  # A perfect rectangle
is_rect, confidence = is_rectangle(points)
print(f"Is Rectangle: {is_rect}, Confidence: {confidence:.2f}%")

points = [[3, 3], [4, 0], [3.9, 3], [0, 3]]  # Slight distortion
is_rect, confidence = is_rectangle(points)
print(f"Is Rectangle: {is_rect}, Confidence: {confidence:.2f}%")

In [10]:
def restructure_array(arr):
    """
    Takes an indivitual set of points and returns the correct format required for the rectangle detector
    """
    return [point[0].tolist() for point in arr]

In [24]:
def straighten_image(src_pts, image):
    """
    Last step: 
    
    """
    src_pts = np.array(restructure_array(src_pts))
    src_pts = src_pts[[0,3,1,2]]
    print(src_pts)
    # Define four destination points (where the corners should be mapped)
    dst_pts = np.array([[0, 0], [0, 200], [200, 200], [200, 0]], dtype=np.float32)
    
    # Compute the homography matrix
    H, _ = cv2.findHomography(src_pts, dst_pts)
    
    # Get the size of the output image
    height, width = 200, 200  # Set the desired output size
    
    # Apply perspective transformation (warp the image)
    warped_image = cv2.warpPerspective(image, H, (width, height))
    return warped_image

In [25]:
def upscale_baby(image):
    """
    Upscale the image using ESR-GAN Lite
    """
    model_path = "/nfs/speed-scratch/z_amm/COEN490/Koala/ALPR/YOLOv8n_TEST/FSRCNN_x4.pb"
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)
    sr.setModel("fsrcnn", 4)  # Use FSRCNN with 4x upscaling
    
    # Apply super-resolution
    upscaled = sr.upsample(image)
    return upscaled